# Configuración Inicial
Este bloque configura las rutas, variables y estructuras necesarias para procesar los datos.


In [39]:
import os
import pandas as pd
import re

# Configuración general
YEAR = 2020
RAW_FOLDER = "../data/raw"
PROCESSED_FOLDER = "../data/processed"
FILE_PATH_XLSX = f"{RAW_FOLDER}/data_{YEAR}.xlsx"
FILE_PATH_XLSB = f"{RAW_FOLDER}/data_{YEAR}.xlsb"
OUTPUT_PATH = f"{PROCESSED_FOLDER}/pluviometros-eda-{YEAR}.csv"

# Mapeo de columnas
COLUMN_MAPPING = {
    'Fecha': ['Fecha', 'Date', 'Día', 'Dia'],
    'Obrador Kiteni': ['Base Operativa Kiteni', 'Kiteni', 'Obrador Kiteni'],
    # Más columnas...
}


# Funciones Auxiliares
Este bloque contiene las funciones necesarias para procesar los datos:
1. Detección de encabezados.
2. Identificación de columnas relevantes.
3. Manejo de valores faltantes (`NaNs`).
4. Agregar información sobre rupturas.
5. Identificar outliers.


In [40]:
def detect_header_row(df, max_rows=15):
    """Detecta la fila que contiene los encabezados reales."""
    for row in range(max_rows):
        possible_header = df.iloc[row].values
        if any(isinstance(val, str) and len(val.strip()) > 0 for val in possible_header):
            return row
    return 0  # Si no se detecta, usar la primera fila

def detect_relevant_columns(df, column_mapping):
    """Detecta columnas relevantes basadas en nombres similares."""
    detected_columns = {}
    for expected_col, possible_names in column_mapping.items():
        for col in df.columns:
            if any(re.search(rf"\b{re.escape(name)}\b", str(col), re.IGNORECASE) for name in possible_names):
                detected_columns[expected_col] = col
                break
    return detected_columns

def handle_missing_values(df, method='global_mean', monthly_averages=None):
    """Reemplaza NaNs con el promedio por columna (global o mensual)."""
    for col in df.columns:
        if col != 'Fecha':
            if method == 'global_mean':
                # Promedio global por columna
                mean_value = df[col].replace(0, pd.NA).mean(skipna=True)
                df[col] = df[col].fillna(mean_value)
            elif method == 'monthly_mean' and monthly_averages is not None:
                # Promedio mensual basado en datos disponibles
                df['Mes'] = pd.to_datetime(df['Fecha']).dt.month
                df[col] = df.apply(
                    lambda row: monthly_averages.loc[row['Mes'], col]
                    if pd.isnull(row[col]) else row[col], axis=1
                )
    return df

def add_rupture_column(df, rupture_dates):
    """Agrega una columna con información sobre rupturas basada en fechas específicas."""
    df['Ruptura'] = df['Fecha'].apply(lambda x: 1 if x in rupture_dates else 0)
    return df

def detect_outliers(df):
    """Detecta días con valores extremos de lluvia basados en el percentil 95."""
    outlier_thresholds = {}
    for col in df.columns:
        if col not in ['Fecha', 'Ruptura', 'Mes']:
            threshold = df[col].quantile(0.95)
            outlier_thresholds[col] = threshold
            print(f"Outlier umbral para {col}: {threshold}")
    return outlier_thresholds


# Carga de Datos
Este bloque carga los datos desde el archivo Excel y consolida todas las hojas.


In [41]:
if os.path.exists(FILE_PATH_XLSX):
    file_path, engine = FILE_PATH_XLSX, None
elif os.path.exists(FILE_PATH_XLSB):
    file_path, engine = FILE_PATH_XLSB, 'pyxlsb'
else:
    raise FileNotFoundError(f"No se encontró archivo para el año {YEAR}")

excel_data = pd.ExcelFile(file_path, engine=engine)
sheet_names = excel_data.sheet_names
print(f"Hojas disponibles: {sheet_names}")

all_processed_data = pd.DataFrame()

if os.path.exists(FILE_PATH_XLSX):
    file_path, engine = FILE_PATH_XLSX, None
elif os.path.exists(FILE_PATH_XLSB):
    file_path, engine = FILE_PATH_XLSB, 'pyxlsb'
else:
    raise FileNotFoundError(f"No se encontró archivo para el año {YEAR}")

excel_data = pd.ExcelFile(file_path, engine=engine)
sheet_names = excel_data.sheet_names
print(f"Hojas disponibles: {sheet_names}")

all_processed_data = pd.DataFrame()

for sheet_name in sheet_names:
    print(f"\nProcesando hoja: {sheet_name}")

    try:
        # Detección de encabezados
        sheet_preview = excel_data.parse(sheet_name, nrows=15)
        header_row = detect_header_row(sheet_preview)
        print(f"Encabezado detectado en la fila: {header_row}")

        # Carga completa de datos
        sheet_data = excel_data.parse(sheet_name, skiprows=header_row)
        print(f"Columnas disponibles antes del mapeo: {sheet_data.columns.tolist()}")
        display(sheet_data.head())  # Ver datos iniciales

        # Eliminar columnas 'Unnamed'
        sheet_data = sheet_data.loc[:, ~sheet_data.columns.str.contains('^Unnamed')]
        print(f"Columnas después de eliminar 'Unnamed': {sheet_data.columns.tolist()}")
        display(sheet_data.head())  # Ver datos después de eliminar 'Unnamed'

        # Renombrar columnas según el mapeo directo
        new_column_names = {}
        for expected_col, possible_names in COLUMN_MAPPING.items():
            for col in sheet_data.columns:
                if any(re.search(rf"\b{re.escape(name)}\b", str(col), re.IGNORECASE) for name in possible_names):
                    new_column_names[col] = expected_col

        # Aplicar el renombramiento
        sheet_data = sheet_data.rename(columns=new_column_names)
        print(f"Columnas después del mapeo: {sheet_data.columns.tolist()}")
        display(sheet_data.head())  # Ver datos después de renombrar

        # Generar columna Fecha
        if 'Fecha' not in sheet_data.columns and 'Dia' in sheet_data.columns:
            sheet_data['Fecha'] = sheet_data['Dia'].apply(
                lambda x: f"{YEAR}-{sheet_name}-{int(x):02d}" if pd.notnull(x) and isinstance(x, (int, float)) else None
            )
            sheet_data.drop(columns=['Dia'], inplace=True)

        print(f"Datos después de generar la columna 'Fecha':")
        display(sheet_data.head())  # Ver datos después de generar 'Fecha'

        # Concatenar datos procesados
        all_processed_data = pd.concat([all_processed_data, sheet_data], ignore_index=True)

    except Exception as e:
        print(f"Error procesando la hoja {sheet_name}: {e}")

print("\nDatos consolidados:")
display(all_processed_data.head())



Hojas disponibles: ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO', 'AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']
Hojas disponibles: ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO', 'AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']

Procesando hoja: ENERO
Encabezado detectado en la fila: 0
Columnas disponibles antes del mapeo: ['Día', 'Base Operativa Kiteni', 'PS-1', 'KP-7+400', 'KP-18', 'KP-30', 'KP-40', 'KP-43+600', 'KP-46+100', 'KP-50', 'KP-57', 'KP-64', 'KP-71', 'KP-82', 'KP-90', 'Unnamed: 15', 'Unnamed: 16', 'KP-95', 'PS-2', 'KP-126', 'KP-181+900', 'KP-184', 'KP-199+800', 'PS-3', 'PS-4']


,Día,Base Operativa Kiteni,PS-1,KP-7+400,KP-18,KP-30,KP-40,KP-43+600,KP-46+100,KP-50,...,Unnamed: 15,Unnamed: 16,KP-95,PS-2,KP-126,KP-181+900,KP-184,KP-199+800,PS-3,PS-4
0,2020-01-01,5.0,73.2,98.0,74.1,82.4,20.3,0,25.2,29.0,...,NaN,NaN,4.4,9.2,6.2,0.0,0.0,0.0,1.0,2.0
1,2020-01-02,41.4,9.2,12.0,29.2,26.6,33.4,0,80.8,55.8,...,NaN,NaN,39.6,11.8,15.0,0.0,0.0,0.0,0.2,0.6
2,2020-01-03,2.8,32.0,71.0,35.3,59.8,18.6,0,28.6,40.4,...,NaN,NaN,4.6,10.6,19.1,0.0,0.0,0.0,0.9,0.8
3,2020-01-04,0.0,0.0,0.0,0.0,33.8,43.4,0,38.2,54.0,...,NaN,NaN,1.2,0.0,3.0,0.0,0.0,0.0,0.4,0.2
4,2020-01-05,13.6,4.4,17.0,19.4,15.2,4.2,0,4.2,0.7,...,NaN,NaN,14.0,24.2,10.0,0.0,0.0,0.0,0.2,0.4


Columnas después de eliminar 'Unnamed': ['Día', 'Base Operativa Kiteni', 'PS-1', 'KP-7+400', 'KP-18', 'KP-30', 'KP-40', 'KP-43+600', 'KP-46+100', 'KP-50', 'KP-57', 'KP-64', 'KP-71', 'KP-82', 'KP-90', 'KP-95', 'PS-2', 'KP-126', 'KP-181+900', 'KP-184', 'KP-199+800', 'PS-3', 'PS-4']


,Día,Base Operativa Kiteni,PS-1,KP-7+400,KP-18,KP-30,KP-40,KP-43+600,KP-46+100,KP-50,...,KP-82,KP-90,KP-95,PS-2,KP-126,KP-181+900,KP-184,KP-199+800,PS-3,PS-4
0,2020-01-01,5.0,73.2,98.0,74.1,82.4,20.3,0,25.2,29.0,...,24.0,20.1,4.4,9.2,6.2,0.0,0.0,0.0,1.0,2.0
1,2020-01-02,41.4,9.2,12.0,29.2,26.6,33.4,0,80.8,55.8,...,5.0,19.3,39.6,11.8,15.0,0.0,0.0,0.0,0.2,0.6
2,2020-01-03,2.8,32.0,71.0,35.3,59.8,18.6,0,28.6,40.4,...,5.0,8.6,4.6,10.6,19.1,0.0,0.0,0.0,0.9,0.8
3,2020-01-04,0.0,0.0,0.0,0.0,33.8,43.4,0,38.2,54.0,...,0.0,0.0,1.2,0.0,3.0,0.0,0.0,0.0,0.4,0.2
4,2020-01-05,13.6,4.4,17.0,19.4,15.2,4.2,0,4.2,0.7,...,6.5,8.4,14.0,24.2,10.0,0.0,0.0,0.0,0.2,0.4


Columnas después del mapeo: ['Fecha', 'Obrador Kiteni', 'PS-1', 'KP-7+400', 'KP-18', 'KP-30', 'KP-40', 'KP-43+600', 'KP-46+100', 'KP-50', 'KP-57', 'KP-64', 'KP-71', 'KP-82', 'KP-90', 'KP-95', 'PS-2', 'KP-126', 'KP-181+900', 'KP-184', 'KP-199+800', 'PS-3', 'PS-4']


...

,Fecha,Obrador Kiteni,PS-1,KP-7+400,KP-18,KP-30,KP-40,KP-43+600,KP-46+100,KP-50,...,KP-82,KP-90,KP-95,PS-2,KP-126,KP-181+900,KP-184,PS-3,PS-4,KP-244
0,2020-12-01,38.0,6.2,10.0,17.2,18.1,48.2,50.0,85.3,36.5,...,28.0,83.0,55.2,55.0,33.4,13.8,43.0,2.6,0.0,0.0
1,2020-12-02,5.2,6.0,6.0,4.1,6.5,2.0,25.2,1.4,7.3,...,2.5,3.9,8.0,69.0,7.8,12.0,17.8,8.8,0.0,6.4
2,2020-12-03,0.0,15.0,6.0,8.1,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.2,8.6,24.2,8.4,0.0,0.0
3,2020-12-04,3.8,70.4,30.0,39.6,23.3,58.5,11.3,24.4,13.9,...,8.1,8.3,0.0,6.0,5.6,17.4,19.6,0.0,0.0,10.6
4,2020-12-05,10.8,53.2,40.0,28.3,36.4,42.2,25.9,16.0,23.5,...,34.1,33.3,24.2,36.6,51.4,6.0,4.0,10.4,0.0,6.6


Datos después de generar la columna 'Fecha':


,Fecha,Obrador Kiteni,PS-1,KP-7+400,KP-18,KP-30,KP-40,KP-43+600,KP-46+100,KP-50,...,KP-82,KP-90,KP-95,PS-2,KP-126,KP-181+900,KP-184,PS-3,PS-4,KP-244
0,2020-12-01,38.0,6.2,10.0,17.2,18.1,48.2,50.0,85.3,36.5,...,28.0,83.0,55.2,55.0,33.4,13.8,43.0,2.6,0.0,0.0
1,2020-12-02,5.2,6.0,6.0,4.1,6.5,2.0,25.2,1.4,7.3,...,2.5,3.9,8.0,69.0,7.8,12.0,17.8,8.8,0.0,6.4
2,2020-12-03,0.0,15.0,6.0,8.1,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.2,8.6,24.2,8.4,0.0,0.0
3,2020-12-04,3.8,70.4,30.0,39.6,23.3,58.5,11.3,24.4,13.9,...,8.1,8.3,0.0,6.0,5.6,17.4,19.6,0.0,0.0,10.6
4,2020-12-05,10.8,53.2,40.0,28.3,36.4,42.2,25.9,16.0,23.5,...,34.1,33.3,24.2,36.6,51.4,6.0,4.0,10.4,0.0,6.6



Datos consolidados:


,Fecha,Obrador Kiteni,PS-1,KP-7+400,KP-18,KP-30,KP-40,KP-43+600,KP-46+100,KP-50,...,KP-95,PS-2,KP-126,KP-181+900,KP-184,KP-199+800,PS-3,PS-4,KP-184+039,KP-244
0,2020-01-01,5.0,73.2,98.0,74.1,82.4,20.3,0.0,25.2,29.0,...,4.4,9.2,6.2,0.0,0.0,0.0,1.0,2.0,NaN,NaN
1,2020-01-02,41.4,9.2,12.0,29.2,26.6,33.4,0.0,80.8,55.8,...,39.6,11.8,15.0,0.0,0.0,0.0,0.2,0.6,NaN,NaN
2,2020-01-03,2.8,32.0,71.0,35.3,59.8,18.6,0.0,28.6,40.4,...,4.6,10.6,19.1,0.0,0.0,0.0,0.9,0.8,NaN,NaN
3,2020-01-04,0.0,0.0,0.0,0.0,33.8,43.4,0.0,38.2,54.0,...,1.2,0.0,3.0,0.0,0.0,0.0,0.4,0.2,NaN,NaN
4,2020-01-05,13.6,4.4,17.0,19.4,15.2,4.2,0.0,4.2,0.7,...,14.0,24.2,10.0,0.0,0.0,0.0,0.2,0.4,NaN,NaN


# Limpieza de Datos y Análisis
Este bloque limpia los datos (manejo de `NaNs`), agrega información de rupturas, detecta outliers y calcula promedios mensuales.


In [42]:
# Calcular promedios mensuales para manejar NaNs
monthly_averages = all_processed_data.groupby(pd.to_datetime(all_processed_data['Fecha']).dt.month).mean(numeric_only=True)

# Manejo de valores faltantes
all_processed_data = handle_missing_values(all_processed_data, method='monthly_mean', monthly_averages=monthly_averages)

# Agregar columna de rupturas
rupture_dates = ['2020-01-15', '2020-05-20']
all_processed_data = add_rupture_column(all_processed_data, rupture_dates)

# Detectar outliers
outlier_thresholds = detect_outliers(all_processed_data)


Outlier umbral para Obrador Kiteni: 23.8
Outlier umbral para PS-1: 48.6
Outlier umbral para KP-7+400: 48.0
Outlier umbral para KP-18: 50.05
Outlier umbral para KP-30: 49.725
Outlier umbral para KP-40: 45.224999999999994
Outlier umbral para KP-43+600: 46.8
Outlier umbral para KP-46+100: 55.449999999999996
Outlier umbral para KP-50: 58.35
Outlier umbral para KP-57: 73.0
Outlier umbral para KP-64: 44.3
Outlier umbral para KP-71: 38.35
Outlier umbral para KP-82: 27.875
Outlier umbral para KP-90: 29.5
Outlier umbral para KP-95: 25.9
Outlier umbral para PS-2: 39.85
Outlier umbral para KP-126: 24.975
Outlier umbral para KP-181+900: 26.0
Outlier umbral para KP-184: 18.1
Outlier umbral para KP-199+800: 1.149999999999966
Outlier umbral para PS-3: 18.6
Outlier umbral para PS-4: 12.75
Outlier umbral para KP-184+039: 0.014999999999999146
Outlier umbral para KP-244: 6.7600000000000025


# Exportar Resultados
Guarda los datos procesados en un archivo CSV consolidado.


In [43]:
all_processed_data.to_csv(OUTPUT_PATH, index=False, encoding='utf-8')
print(f"Datos consolidados guardados en: {OUTPUT_PATH}")


Datos consolidados guardados en: ../data/processed/pluviometros-eda-2020.csv
